In [1]:
import time
start = time.time()
from processing import train_data, embed
end = time.time()
print(end - start)
# from multi_processing import train_data 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231903 entries, 0 to 260086
Data columns (total 5 columns):
embedded_state     231903 non-null object
state              231903 non-null object
embedded_action    231903 non-null object
action             231903 non-null object
reward             231903 non-null object
dtypes: object(5)
memory usage: 242.8 MB
None
20.501777172088623


In [2]:
import numpy as np
from random import randint
import pandas as pd
from functools import partial

## Environment

In [54]:
class itemEnv():
    def __init__(self, alpha = 0.5, sigma = 0.9): # albha parameter in consine similarity
        self.observation_space = train_data
        self.item_info = embed
        self.current_state = None
        self.init_state = self.reset()
        self.sigma = sigma
        self.alpha = alpha
        self.embedding_dim = 7
        self.reward_val = 1
        self.action_num = 9
#         self.rewards, self.avg_state, self.avg_action, self.nx_size = self.avg_group()
    
    def reset(self):
        init_state = self.observation_space['embedded_state'].sample(1).values[0]
#         init_state = self.observation_space.loc[0,'embedded_state']
        self.current_state = init_state
        return init_state

#     def feedback(self, pair):  # pair: state-action pair
#         prob = list()
#         denominator = 0.
#         max_prob = 0.
#         result = 0.
#         feed_back = ""
#         for r, s, a in zip(self.rewards, self.avg_state, self.avg_action):
#             numerator = np.dot(pair[0], s.T) / np.linalg.norm(pair[0], 2) + np.dot(pair[1], a.T) / np.linalg.norm(pair[1], 2)
#             denominator += numerator
#             prob.append(numerator)
#             if numerator > max_prob:
#                 max_prob = numerator
#                 feed_back = r
#         prob = prob/denominator
# #         for p, r in zip(prob, self.rewards):
# #             for k in range(1):
# #                 result += p * self.reward_val * np.power(self.sigma, k) * int(r.split(',')[k])
        
#         result = self.reward_val * int(feed_back.split(',')[0])
#         print(prob)
#         return feed_back, result

    @staticmethod
    def cos_sim(group, pair):
        max_prob = 0.
        # nx_size = len(group[1])
        s_i = np.array(group[1]['embedded_state'].values.tolist()) # 2 dim array:0-->sample, 1-->feature_num (N,(7*12))
        s_t = pair[0]                                              # 1 dim array:0-->sample, (7*12,)
#         a_i = np.array(group[1]['embedded_action'].values.tolist()) #(N,9,7)
        a_i = np.reshape(np.array(group[1]['embedded_action'].values.tolist()),[-1, env.action_num * env.embedding_dim]) #(N, 9*7)
        
        a_t = np.reshape(pair[1], [env.action_num * env.embedding_dim])                                               #(9*7,)
        norm_si = np.linalg.norm(s_i, 2, axis = 1)                 # 1 dim (N,)
        norm_st = np.linalg.norm(s_t, 2, axis = 0)                 # 1 dim (1,)
        norm_ai = np.linalg.norm(a_i, 2, axis = 1)                 #1 dim (N,)
        norm_at = np.linalg.norm(a_t, 2, axis = 0)                 # 1 dim (1,)
        
        #first term: (N,), second term: (N,)
        cos = env.alpha * (np.dot(s_i, s_t)/(norm_si * norm_st + 1e-10)) + (1 - env.alpha) * np.dot(a_i, a_t)/(norm_ai * norm_at + 1e-10)  #1 dim
        # cos = cos / np.sum(cos)
        #cos: (N,)
        # max_id = np.argmax(cos)
#         id_x = int(max_id / cos.shape[1])
#         id_y = max_id % cos.shape[1]
        # print('i am here!')
        return np.max(cos)
    
    def feedback_2(self, pair):
        data = train_data.groupby(['reward'])
#         denominator = 0.
        reward_ind = -1
        max_prob = 0.
        result = 0.
        feed_back = ""
        fnc = partial(self.cos_sim, pair = pair)
        prob = map(fnc, data)
        r = 0.
        for tup, group in zip(prob, data):
            if tup > max_prob:
                max_prob = tup
#                 reward_ind = tup[2]  #id_y
                feed_back = group[0]          #index: reward str
#                 feed_back = group[1].iloc[tup[1], 2]
#                 idx = tup[1]
        for k, reward in enumerate(feed_back.split(',')):
            r += max_prob * np.power(self.sigma, k) * (-1 if reward == "0" else 1)
            
#         r = 1 * int(feed_back.split(','))  #reward_val
#         result =  r if r > 0 else -1

        return feed_back, r, max_prob
            

    def step(self, action):   #action: 2 dim array
        feed_back, result, p = self.feedback_2((self.current_state, action))
#         for i,r in enumerate([feed_back.split(',')[0]]): #0---> one action
        for i, r in enumerate(feed_back.split(',')):
            if r != "0":
                # self.current_state.append(action[i])
                tmp = np.append(self.current_state, action[i,:])
#                 tmp = np.delete(tmp, 0, axis=0)
                tmp = tmp[self.embedding_dim:]
                self.current_state = tmp

#         if result == 1:     #reward_val
#             tmp = np.append(self.current_state, action)
#             tmp = tmp[self.embedding_dim:]
#             self.current_state = tmp  # state: 1 dim array
# #         else:
# #             self.current_state = self.observation_space['embedded_state'].sample(1).values[0]

        return result, self.current_state, feed_back, p

#     def avg_group(self):
#         nx_size = list()
#         avg_state = list()
#         avg_action = list()
#         rewards = list()
#         for reward, group in self.observation_space.groupby(['reward']):
#             nx_size.append(group.shape[0])
# #             state = np.mean(data['embedded_action'], axis = 0)
#             norm_s = np.linalg.norm(np.array(group['embedded_state'].values.tolist()), 2, axis = 1)
#             norm_s = np.where(norm_s == 0, 0.001, norm_s)
#             state = np.sum(group['embedded_state'] / norm_s) / group.shape[0]
#             norm_a = np.linalg.norm(np.array(group['embedded_action'].values.tolist()), 2, axis = 1)
#             norm_a = np.where(norm_a == 0, 0.001, norm_a)
#             action = np.sum(group['embedded_action'] / norm_a) / group.shape[0]
#             avg_state.append(state)
#             avg_action.append(action)
#             rewards.append(reward)

#         return rewards, avg_state, avg_action, nx_size
def gen_action(item_id):
    return embed[item_id]



In [55]:
env = itemEnv()
env.reset()
env.current_state

array([ 1.87232473, -0.2268461 , -0.72387844, -0.11876125,  0.61762612,
       -0.74414196,  0.72406985,  1.87232473, -0.61097216, -0.72387844,
       -0.11926497,  0.36961457, -0.77438383,  0.72406985, -0.82978028,
        1.69378421, -0.72387844, -0.10210977,  0.60668444,  0.92300988,
        0.72406985,  0.07092139, -0.61097216, -0.72387844, -0.1151329 ,
        1.83580053, -1.0553583 , -2.24228081,  0.97162306, -0.61097216,
       -0.72387844, -0.12482077,  1.38354416, -0.80352599, -2.24228081,
        0.97162306, -0.61097216, -0.72387844, -0.12244425,  1.31424681,
       -0.67211132, -2.24228081,  0.07092139, -0.2268461 , -0.72387844,
       -0.12003396,  1.53308054, -0.66001457, -0.75910548,  1.87232473,
       -0.61097216, -0.72387844, -0.12301752,  1.02611574, -0.47086542,
       -0.75910548,  0.97162306,  2.07791027, -0.72387844, -0.12267674,
        1.15376875, -0.17504494, -0.75910548,  1.87232473,  0.92553209,
       -0.72387844, -0.11692224,  0.96776008, -0.8805053 ,  0.72

In [56]:

start = time.time()
# action = train_data['embedded_action'].sample(1).values[0][5]
# action = train_data.loc[0,'embedded_action'][0]
action_l = []
for i in range(9):
    action_l.append(str(randint(1, 381)))

action = np.array(list(map(gen_action, action_l)))
r,s,f,p = env.step(action)
end = time.time()
r, s, f,p,end - start
# action, action_l

(1.968528482739641,
 array([ 1.87232473, -0.61097216, -0.72387844, -0.12301752,  1.02611574,
        -0.47086542, -0.75910548,  0.97162306,  2.07791027, -0.72387844,
        -0.12267674,  1.15376875, -0.17504494, -0.75910548,  1.87232473,
         0.92553209, -0.72387844, -0.11692224,  0.96776008, -0.8805053 ,
         0.72406985,  0.97162306, -0.61097216, -0.72387844, -0.12460815,
        -0.25406155, -0.63527122,  0.72406985, -0.82978028,  1.69378421,
        -0.72387844, -0.10210977,  0.60668444,  0.92300988,  0.72406985,
        -0.82978028, -0.61097216,  1.38144741, -0.12759072, -0.99444899,
         0.60574517,  0.72406985,  0.07092139, -0.2268461 , -0.72387844,
        -0.12688829, -0.37806733, -0.57588719,  0.72406985, -0.82978028,
         2.07791027, -0.72387844, -0.12561956, -0.87409044,  1.99027298,
         0.72406985, -0.82978028, -0.61097216, -0.72387844, -0.12327285,
         0.7926931 , -0.9563849 , -0.75910548,  0.97162306, -0.2268461 ,
        -0.72387844, -0.1224611

In [ ]:
start = time.time()
action_l = []
for i in range(1):
    action_l.append(str(randint(1, 381)))

action = np.array(list(map(gen_action, action_l)))
r,s,f,p = env.step(action)
end = time.time()
r,s,f,p,end-start

In [57]:
train_data.groupby('reward').size()

reward
0,0,0,0,0,0,0,0,0    17548
0,0,1,0,0,0,0,0,0     4749
0,1,0,0,0,0,0,0,0     4576
0,1,1,0,0,0,0,0,0     2487
1,0,0,0,0,0,0,0,0     5481
1,0,1,0,0,0,0,0,0     2662
1,1,0,0,0,0,0,0,0     3555
1,1,0,0,1,0,0,0,1        1
1,1,1,0,0,0,0,0,0     3918
1,1,1,0,0,1,0,0,0     3644
1,1,1,0,1,0,0,0,0     3557
1,1,1,0,1,1,0,0,0     2806
1,1,1,1,0,0,0,0,0     4175
1,1,1,1,0,1,0,0,0     2983
1,1,1,1,1,0,0,0,0     4286
1,1,1,1,1,1,0,0,0     2877
1,1,1,1,1,1,0,0,1     4394
1,1,1,1,1,1,0,1,0     7328
1,1,1,1,1,1,0,1,1     6496
1,1,1,1,1,1,1,0,0     9277
1,1,1,1,1,1,1,0,1     8031
1,1,1,1,1,1,1,1,0    31676
1,1,1,1,1,1,1,1,1    25825
dtype: int64

## Network

In [61]:
import torch
from torch import nn
from torch.nn import functional as act
from torch.distributions import Categorical
from torch.autograd import Variable

In [62]:
def try_gpu():
    i = 0
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

try_gpu()

device(type='cuda', index=0)

In [63]:
class LSTM(nn.Module):
    def __init__(self, inpt_sz, hidden_sz, hidden_l1, oupt_sz): #embedding dim, , , 381
        super().__init__()
        #input_gate
        device = try_gpu()
        init_weights = partial(self.init_, device = device)
#         self.init_state = None
        self.reward = []
        self.inpt_sz = inpt_sz
        self.hidden_sz = hidden_sz
        self.hidden_l1 = hidden_l1
        self. oupt_sz = oupt_sz
        self.W_xi, self.W_hi, self.bias_i = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #forget_gate
        self.W_xf, self.W_hf, self.bias_f = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #output_gate
        self.W_xo, self.W_ho, self.bias_o = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #candidate memory cell
        self.W_xc, self.W_hc, self.bias_c = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #output_layer
        self.W_o1, self.W_o2, self.bias_o1, self.bias_o2 = (init_weights((hidden_sz, hidden_l1)), init_weights(
                                            (hidden_l1, oupt_sz)), init_weights((1, hidden_l1)), 
                                            init_weights((1, oupt_sz))
                                            ) 
    @staticmethod
    def init_(shape, device):
#         param = torch.tensor(shape)
        def xvaier(param):
            return nn.init.xavier_uniform_(param)
        param = xvaier(torch.rand(shape, device = device))
        return nn.Parameter(param)
    
    def forward(self, X, init_state = None):  #X: batch size, seq size, input size
        batch_size, seq_size, _ = X.shape
        hidden_sz = self.hidden_sz
#         oupts = []
        if init_state == None:         #H, C actually are constant, not trainable
            H, C = (torch.zeros(batch_size, hidden_sz, device = X.device),
                    torch.zeros(batch_size, hidden_sz, device = X.device)
                   )
        else:
            H, C = init_state    # in some circumstance
#         softmax = act.softmax(dim = 1)
        for seq in range(seq_size):
            x_t = X[:,seq,:]
            I = torch.sigmoid(x_t @ self.W_xi + H @ self.W_hi + self.bias_i)
            F = torch.sigmoid(x_t @ self.W_xf + H @ self.W_hf + self.bias_f)
            O = torch.sigmoid(x_t @ self.W_xo + H @ self.W_ho + self.bias_o)
            C_tilda = torch.tanh(x_t @ self.W_xc + H @ self.W_hc + self.bias_c)
            
            C = F * C + I * C_tilda
            H = torch.tanh(C) + O
            hidden_layer1 = torch.relu(H @ self.W_o1 + self.bias_o1)
            output = act.softmax(hidden_layer1 @ self.W_o2 + self.bias_o2, dim = 1)
#             oupts.append(output)
            
#         self.init_state = (H, C)
#         return torch.cat(oupts, dim = 0)
        return output, (H, C)
        
        
    
    

In [64]:
actor = LSTM(7, 256, 256, 381)

In [75]:
n_state = np.reshape(env.current_state, [-1, 12, 7])
n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
# bat_x.append(n_state)
prob, _= actor(n_state)
m = Categorical(prob)
m.sample((3,)).squeeze(dim = 1)

tensor([182, 330, 336], device='cuda:0')

In [65]:
def action_list(m):
    action_l = []
    action_l.append(str(m.sample().item()))
    for i in range(1, 9, 1):
        while True:
            action = str(m.sample.item())
            if action == action_l:
        action_l.append(str(m.sample.item()))

    actions = np.array(list(map(gen_action, action_l)))
    return actions

SyntaxError: invalid syntax (<ipython-input-65-cdc9f71c323c>, line 7)

In [42]:
#train
# def maximumll(inpt, label):
    
start = time.time()
    
loss_fn = nn.NLLLoss(reduction='sum')
max_episodes = 100
max_episodes_len = 16
gamma = 0.8
optimizer = torch.optim.SGD(actor.parameters(), lr = 0.1)
for episode in range(max_episodes):
    env.reset()
    G_t = 0.
    bat_x = []
    target = []
    policy_loss_l = []
#     del actor.reward[:]
    for i in range(max_episodes_len):
        n_state = np.reshape(env.current_state, [-1, 12, 7])
        n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
        bat_x.append(n_state)
        prob, _= actor(n_state)
        
        m = Categorical(prob)

#         action = m.sample()
        actions = m.sample((9,)).squeeze(dim = 1)
#         target.append(action)
        embed_action = gen_action(str(action.item()))
        r, _, _ = env.step(embed_action)
#         actor.reward.append(r)
        G_t = gamma*G_t + r
        policy_loss_l.append(-m.log_prob(action) * G_t)
    
#     bat_x = torch.cat(bat_x, 0)
#     target = torch.cat(target, -1)
    loss = torch.cat(policy_loss_l).sum()  
    loss.backward()
#     pred, _ = actor(bat_x)
#     log_prob = torch.log(pred)
#     loss = loss_fn(G_t * log_prob, target)
    
#     loss.backward()
    optimizer.step()
    
    print(f'total reward:{G_t}, loss:{loss.data}')

end = time.time()
end - start
# aa = torch.cat(aa, 0)

total reward:-2.2727415814225926, loss:-175.91790771484375
total reward:-3.0435854883553275, loss:0.32864493131637573
total reward:-4.700932837244929, loss:-4.910978532279842e-06
total reward:-4.700932837244929, loss:-4.910978532279842e-06
total reward:-3.7985602083553283, loss:6.192017281136941e-07
total reward:-4.453542720995328, loss:-2.6447573873156216e-06
total reward:-4.281744029155329, loss:-1.5345842712122248e-06
total reward:-4.453542720995328, loss:-2.6447573873156216e-06


KeyboardInterrupt: 

In [22]:
loss = nn.NLLLoss()
a = torch.Tensor([[1,2,3]])
b = torch.Tensor([[1, 0, 0]])
c = torch.Tensor([0]).long()

o = act.softmax(a, dim = 1)
y = -o.log() @ b.T
p = loss(o.log(), c)
y,p

(tensor([[2.4076]]), tensor(2.4076))

In [45]:
s = train_data.loc[0,'embedded_state']
n_state = np.reshape(s, [-1, 12, 7])
n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
p,_=actor(n_state)
torch.argmax(p),p

(tensor(35, device='cuda:0'),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [46]:
train_data.loc[0,'action']

[3, 28, 20, 84, 105, 106, 239, 212, 151]

In [12]:
pred

tensor([[0.0013, 0.0010, 0.0039, 0.0020, 0.0048, 0.0019, 0.0030, 0.0011, 0.0011,
         0.0051, 0.0008, 0.0033, 0.0086, 0.0026, 0.0011, 0.0008, 0.0040, 0.0034,
         0.0029, 0.0018, 0.0053, 0.0014, 0.0024, 0.0010, 0.0013, 0.0018, 0.0069,
         0.0005, 0.0012, 0.0030, 0.0027, 0.0023, 0.0013, 0.0024, 0.0019, 0.0024,
         0.0016, 0.0057, 0.0013, 0.0007, 0.0094, 0.0005, 0.0023, 0.0015, 0.0034,
         0.0050, 0.0019, 0.0018, 0.0071, 0.0021, 0.0023, 0.0024, 0.0014, 0.0010,
         0.0029, 0.0007, 0.0020, 0.0045, 0.0032, 0.0016, 0.0013, 0.0019, 0.0033,
         0.0081, 0.0018, 0.0013, 0.0022, 0.0028, 0.0023, 0.0014, 0.0029, 0.0018,
         0.0021, 0.0034, 0.0022, 0.0021, 0.0016, 0.0011, 0.0008, 0.0029, 0.0036,
         0.0026, 0.0064, 0.0030, 0.0009, 0.0018, 0.0030, 0.0063, 0.0027, 0.0020,
         0.0032, 0.0022, 0.0044, 0.0040, 0.0008, 0.0016, 0.0056, 0.0006, 0.0030,
         0.0017, 0.0042, 0.0071, 0.0014, 0.0034, 0.0027, 0.0077, 0.0007, 0.0013,
         0.0027, 0.0023, 0.0

In [13]:
G_t * pred

tensor([[-161.3674, -155.0946, -156.8075,  ..., -153.8318, -187.5206,
         -160.8493],
        [-157.4997, -150.8772, -152.9312,  ..., -150.8477, -182.5345,
         -156.8122],
        [-156.8636, -150.4019, -152.0855,  ..., -150.4251, -181.2172,
         -156.2092],
        ...,
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119],
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119],
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119]], device='cuda:0', grad_fn=<MulBackward0>)

In [13]:
target

tensor([358, 307], device='cuda:0')

In [45]:
actor.parameters()

<generator object Module.parameters at 0x0000023CC7FCE480>

In [ ]:
def compare(group, x):
    max_prob = 0.
    nx_size = len(group)
    
    for i in range(nx_size):
        tmp = group.loc[i,'two'] + x
        if tmp > max_prob:
            max_prob = tmp
    print(max_prob)
    return max_prob
        
    
pool = ProcessPoolExecutor(max_workers=8)
for _ in pool.map(compare, (aa.groupby('one'), 1)):
    pass

In [55]:
for group in aa.groupby('one'):   
    print(group[1])
aa.groupby('one').size()

  one              two  three
0   1  [1.0, 1.0, 1.0]      3
2   1  [0.0, 0.0, 0.0]      5
  one              two  three
1   2  [0.0, 0.0, 0.0]      4
3   2  [1.0, 1.0, 1.0]      8


one
1    2
2    2
dtype: int64

In [67]:
for group in aa.groupby('one'):   
    i = np.argmax(group[1]['three'].values.tolist())
    print(i)
    print(group[1].iloc[i,1])

1
[1.5 1.5 1.5]
0
[0. 0. 0.]


In [43]:
for group in aa.groupby('one'):
    print(np.dot(np.array(group[1]['two'].values.tolist()), np.ones(3)))


[3. 0.]
[0. 3.]


In [44]:
for group in aa.groupby('one'):
    print(np.array(group[1]['two'].values.tolist()))

[[1. 1. 1.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 1. 1.]]


In [48]:
for group in aa.groupby('one'):
    a = np.array(group[1]['two'].values.tolist())
    n = np.linalg.norm(a,2,axis = 1)
    d = np.dot(a, np.ones(3))
    print(d/n)

[1.73205081        nan]
[       nan 1.73205081]


D:\Applications\Anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [14]:
from multiprocessing import Pool
import long_time_task
import os

In [42]:
dd = np.array([[1,2,3],[4,5,6]])
np.max(dd)

6

In [ ]:
norm_2 = np.linalg.norm(np.array(train_data['embedded_action'].values.tolist()), 2, axis = 1)
norm_2 = np.where(norm_2 == 0, 0.001, norm_2)
norm_2.shape

In [ ]:
np.linalg.norm(train_data.loc[0,'embedded_state'],2)

In [ ]:
np.sum(train_data['embedded_action'] / norm_2)

In [ ]:
bb = train_data['embedded_action'] / norm_2

In [ ]:
np.dot(bb[5], bb[5].T)

In [ ]:
bb.shape

In [ ]:
np.array([[],2,3])

In [ ]:
print(train_data.info(memory_usage = 'deep'))

In [ ]:
# train_data['reward'] = train_data['reward'].apply(lambda row: str(row))
group_data = train_data.groupby('reward')  
for index, data in group_data:
    print(type(index))
#     print(j)

In [ ]:
group_data.size()